In [5]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import torch
import numpy as np
os.environ['TORCH'] = torch.__version__
os.environ['DGLBACKEND'] = "pytorch"


In [4]:
pip install dgl -f https://data.dgl.ai/wheels/torch-2.4/repo.html

Looking in links: https://data.dgl.ai/wheels/torch-2.4/repo.html
  Using cached torchdata-0.9.0-cp312-cp312-win_amd64.whl.metadata (5.6 kB)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached torch-2.5.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
Using cached torchdata-0.9.0-cp312-cp312-win_amd64.whl (1.4 MB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached torch-2.5.1-cp312-cp312-win_amd64.whl (203.0 MB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
Using cached fsspec-2024.10.0-py3-none-any.whl (179 kB)
   ---------------------------------------- 0.0/53

In [6]:

os.environ['TORCH'] = torch.__version__
os.environ['DGLBACKEND'] = "pytorch"
import dgl
import dgl.graphbolt as gb
import torch.nn as nn
import torch.nn.functional as F
import sklearn.metrics
device = torch.device("cpu")

FileNotFoundError: Cannot find DGL C++ graphbolt library at c:\Users\MEKhani\anaconda3\envs\dglenv\Lib\site-packages\dgl\graphbolt\graphbolt_pytorch_2.5.1.dll

Loading Dataset

In [3]:
dataset = gb.BuiltinDataset("ogbn-arxiv",root="../../../data/datasets").load()

The dataset is already preprocessed.


Dataset consists of graph, feature and tasks.

In [4]:
graph = dataset.graph
feature = dataset.feature
train_set = dataset.tasks[0].train_set
valid_set = dataset.tasks[0].validation_set
test_set = dataset.tasks[0].test_set
task_name = dataset.tasks[0].metadata["name"]
num_classes = dataset.tasks[0].metadata["num_classes"]
print(f"Task: {task_name}. Number of classes: {num_classes}")

Task: node_classification. Number of classes: 40


Defining Neighbor Sampler and Data Loader in DGL

In [5]:
datapipe = gb.ItemSampler(train_set, batch_size=1024, shuffle=True)
datapipe = datapipe.sample_neighbor(graph, [4, 4])
datapipe = datapipe.fetch_feature(feature, node_feature_keys=["feat"])
datapipe = datapipe.copy_to(device)
train_dataloader = gb.DataLoader(datapipe, num_workers=0)

iterate over the data loader 

In [6]:
data = next(iter(train_dataloader))
print(data)
print("hello to new envirement ")

MiniBatch(seeds=tensor([ 58827, 158930,  33891,  ...,  40174,  87576, 101915]),
          sampled_subgraphs=[SampledSubgraphImpl(sampled_csc=CSCFormatBase(indptr=tensor([    0,     4,     7,  ..., 14817, 14819, 14823], dtype=torch.int32),
                                                                         indices=tensor([ 3886,  1026,  3593,  ...,  3527,  5759, 11912], dtype=torch.int32),
                                                           ),
                                               original_row_node_ids=tensor([ 58827, 158930,  33891,  ..., 124995, 109848, 145819]),
                                               original_edge_ids=tensor([ 964979, 1482575, 2079753,  ...,  816651, 1765725, 1999714]),
                                               original_column_node_ids=tensor([ 58827, 158930,  33891,  ..., 103509,  28854,  18145]),
                            ),
                            SampledSubgraphImpl(sampled_csc=CSCFormatBase(indptr=tensor([   0,    4,    7,

 node IDs from MFGs

In [7]:
mfgs =data.blocks
input_nodes = mfgs[0].srcdata[dgl.NID]
print(f"Input nodes {input_nodes }. ")

Input nodes tensor([ 58827, 158930,  33891,  ..., 124995, 109848, 145819]). 


defining model

In [8]:
from dgl.nn import SAGEConv

class myModel(nn.Module):
    def __init__(self ,in_feat,h_feat,num_classes):
        super().__init__()
        self.conv1 = SAGEConv(in_feat,h_feat,aggregator_type="mean")
        self.conv2 = SAGEConv(h_feat,num_classes,aggregator_type="mean")
        self.h_feat = h_feat 
    def forward(self ,mfgs,x):
        h= self.conv1(mfgs[0],x)
        h = F.relu(h)
        h = self.conv2(mfgs[1],h)
        return h 
in_size = feature.size("node", None, "feat")[0]
model = myModel(in_size,16,num_classes)

train model

In [9]:
optimizer = torch.optim.Adam(model.parameters())

define valdiation loader 

In [10]:
datapipe = gb.ItemSampler(valid_set, batch_size=1024, shuffle=True)
datapipe = datapipe.sample_neighbor(graph, [4, 4])
datapipe = datapipe.fetch_feature(feature, node_feature_keys=["feat"])
datapipe = datapipe.copy_to(device)
valid_dataloader = gb.DataLoader(datapipe, num_workers=0)

In [11]:
import tqdm 

for epoch in range(100):
    model.train()
    with tqdm.tqdm(train_dataloader) as tq:
        for step , data in enumerate(tq):
            x = data.node_features["feat"]
            labels = data.labels

            predictions= model(data.blocks,x)

            loss = F.cross_entropy(predictions,labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            accuracy = sklearn.metrics.accuracy_score(
                labels.cpu().numpy(),predictions.argmax(1).detach().cpu().numpy()
            )
            tq.set_postfix(
                 {"loss": "%.03f" % loss.item(), "acc": "%.03f" % accuracy},
                refresh=False,
            )
    model.eval()
    predictions = []
    labels = []
    with tqdm.tqdm(valid_dataloader) as tq:
        for step , data in enumerate(tq):
            x = data.node_features["feat"]
            labels.append(data.labels.cpu().numpy())
            predictions.append(model(data.blocks, x).argmax(1).cpu().numpy())
        predictions = np.concatenate(predictions)
        labels = np.concatenate(labels)
        accuracy = sklearn.metrics.accuracy_score(labels, predictions)
        print("Epoch {} Validation Accuracy {}".format(epoch, accuracy))

        # Note that this tutorial do not train the whole model to the end.
    
        break

0it [00:00, ?it/s]

89it [00:02, 33.41it/s, loss=2.858, acc=0.267]
30it [00:00, 48.64it/s]

Epoch 0 Validation Accuracy 0.3102117520722172


Link Prediction

Loading Dataset